In [53]:
import pandas as pd
import numpy as np
import math

In [252]:
model = "m1"

In [253]:
df = pd.read_csv('test_data_'+model+'_clean.csv')
df.head(2)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Random ID,CRIS ID,Document ID,Domain,Date,Text,length,...,document_length,ContextAroundSpan,start_index,end_index,SpanCat_Predictions,SpanCat_Predictions_copy,span1,label1,span2,label2
0,0,0,3,0.00462,10008182,24967795_Comments_Event,3,24/03/2010 00:00,she was unable to attend her literacy class fo...,88,...,986,time she discussed her sexual abuse at the ha...,200,287,[],[],NaN,NaN,NaN,NaN
1,1,1,23,0.00462,10008182,24918548_Comments_Event,3,19/03/2010 00:00,arnie also said zzzzz was schedule to do volun...,64,...,540,spoke with arnie senior cos worker who gave m...,200,263,[],[],NaN,NaN,NaN,NaN


In [254]:
df['label'].value_counts()

label
3and1    202
3and0     65
Name: count, dtype: int64

In [255]:
df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Random ID', 'CRIS ID',
       'Document ID', 'Domain ', 'Date ', 'Text', 'length', 'Code', 'label',
       'Self-harm', 'Suicide', 'Physical', 'Historic',
       'Dysfunctional social relationship', 'Notes',
       'Time taken to code (hours)', 'CN_Doc_ID', 'Brcid', 'Cn_Doc_id',
       'src_col', 'src_table', 'date', 'TextContent', 'is_substring_fuzzy',
       'StartIndex', 'EndIndex', 'document_length', 'ContextAroundSpan',
       'start_index', 'end_index', 'SpanCat_Predictions',
       'SpanCat_Predictions_copy', 'span1', 'label1', 'span2', 'label2'],
      dtype='object')

In [256]:
df['TextContent'] = df['TextContent'].astype(str)
df['Text'] = df['Text'].astype(str)
df['span1'] = df['span1'].astype(str)

df['label'] = df['label'].astype(str)
df['label1'] = df['label1'].astype(str)

In [257]:
def match_type(gold, pred):
    if pd.isna(pred) or pred.strip() == '' or pred=="nan":
        return "no pred (FN)"
    elif pred in gold:
        if pred == gold:
            return "complete"
        else:
            return "partial"
    else:
        return "no match"

df['match'] = df.apply(lambda row: match_type(row['Text'], row['span1']), axis=1)

In [258]:
df.head(2)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Random ID,CRIS ID,Document ID,Domain,Date,Text,length,...,ContextAroundSpan,start_index,end_index,SpanCat_Predictions,SpanCat_Predictions_copy,span1,label1,span2,label2,match
0,0,0,3,0.00462,10008182,24967795_Comments_Event,3,24/03/2010 00:00,she was unable to attend her literacy class fo...,88,...,time she discussed her sexual abuse at the ha...,200,287,[],[],nan,nan,NaN,NaN,no pred (FN)
1,1,1,23,0.00462,10008182,24918548_Comments_Event,3,19/03/2010 00:00,arnie also said zzzzz was schedule to do volun...,64,...,spoke with arnie senior cos worker who gave m...,200,263,[],[],nan,nan,NaN,NaN,no pred (FN)


In [259]:
map_metrics = {
    'partial': 'TP',
    'complete': 'TP',
    'no pred (FN)': 'FN',
    'no match': 'FP'
}

df['span_metrics'] = df['match'].map(map_metrics)

df.head(2)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Random ID,CRIS ID,Document ID,Domain,Date,Text,length,...,start_index,end_index,SpanCat_Predictions,SpanCat_Predictions_copy,span1,label1,span2,label2,match,span_metrics
0,0,0,3,0.00462,10008182,24967795_Comments_Event,3,24/03/2010 00:00,she was unable to attend her literacy class fo...,88,...,200,287,[],[],nan,nan,NaN,NaN,no pred (FN),FN
1,1,1,23,0.00462,10008182,24918548_Comments_Event,3,19/03/2010 00:00,arnie also said zzzzz was schedule to do volun...,64,...,200,263,[],[],nan,nan,NaN,NaN,no pred (FN),FN


In [260]:
df['span_metrics'].value_counts()

span_metrics
FN    219
TP     25
FP     23
Name: count, dtype: int64

In [261]:
len(df)

267

There are some blanks where document_length is blank but i am not able to delete them here

So download the file, delete them and re-upload

Done for m6; m1, m2, m3, m4 and m5 were fine and didn't need it

In [262]:
df.to_csv('evaluation_metrics_'+model+'_CI_v3.csv', index=False)

In [44]:
df = pd.read_csv('evaluation_metrics_'+model+'_CI_v3_clean.csv')

In [263]:
len(df)

267

In [264]:
df['span_metrics'].value_counts()

span_metrics
FN    219
TP     25
FP     23
Name: count, dtype: int64

In [265]:
count_metrics = df['span_metrics'].value_counts()

In [266]:
TP = count_metrics.get('TP', 0)
FN = count_metrics.get('FN', 0)
FP = count_metrics.get('FP', 0)
TN = count_metrics.get('TN', 0)

print("TP: ", TP)
print("FN: ", FN)
print("FP: ", FP)
print("TN: ", TN)

TP:  25
FN:  219
FP:  23
TN:  0


In [267]:
precision = TP/(TP+FP)
recall = TP/(TP+FN)
f1 = 2*(precision*recall)/(precision+recall)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1)

Precision:  0.5208333333333334
Recall:  0.10245901639344263
F1-score:  0.17123287671232879


In [268]:
SE_precision = math.sqrt((precision*(1-precision)/(TP+FP)))
SE_recall = math.sqrt((recall*(1-recall)/(TP+FN)))

print("Precision std error: ", SE_precision)
print("Recall std error: ", SE_recall)

Precision std error:  0.07210610992116384
Recall std error:  0.019413655658773326


In [269]:
z = 1.96

In [270]:
precision_lower_ci = precision - (1.96*SE_precision)
precision_upper_ci = precision + (1.96*SE_precision)

recall_lower_ci = recall - (1.96*SE_recall)
recall_upper_ci = recall + (1.96*SE_recall)

f1_lower_ci = 2*(precision_lower_ci*recall_lower_ci)/(precision_lower_ci+recall_lower_ci)
f1_upper_ci = 2*(precision_upper_ci*recall_upper_ci)/(precision_upper_ci+recall_upper_ci)

print("Precision lower CI: ", precision_lower_ci)
print("Precision upper CI: ", precision_upper_ci)
print("Recall lower CI: ", recall_lower_ci)
print("Recall lower CI: ", recall_upper_ci)
print("F1 lower CI: ", f1_lower_ci)
print("F1 lower CI: ", f1_upper_ci)

Precision lower CI:  0.3795053578878522
Precision upper CI:  0.6621613087788145
Recall lower CI:  0.06440825130224691
Recall lower CI:  0.14050978148463833
F1 lower CI:  0.11012627662389364
F1 lower CI:  0.23182631574174598


In [271]:
model_metrics = {
    'Metric': ['F1-score', 'Precision', 'Recall'],
    'Value': [f1, precision, recall],
    'Lower 95% CI': [f1_lower_ci, precision_lower_ci, recall_lower_ci],
    'Upper 95% CI': [f1_upper_ci, precision_upper_ci, recall_upper_ci]
}

In [272]:
df_metrics = pd.DataFrame(model_metrics)
df_metrics

,Metric,Value,Lower 95% CI,Upper 95% CI
0,F1-score,0.171233,0.110126,0.231826
1,Precision,0.520833,0.379505,0.662161
2,Recall,0.102459,0.064408,0.140510


In [273]:
df_metrics.to_csv('manual_eval_metrics_span_'+model+'_.csv')

# For Labels

In [201]:
df.head(2)

,Unnamed: 0.1,Unnamed: 0,Random ID,CRIS ID,Document ID,Domain,Date,Text,length,Code,...,SpanCat_Predictions,SpanCat_Predictions_copy,span1,label1,span2,label2,span3,label3,match,span_metrics
0,0,1,0.003907,10102728,67051173_Comments_Event,4,16/06/2021 00:00,appeared slightly dishevelled,30,0,...,[],[],nan,nan,NaN,NaN,NaN,NaN,no pred (FN),FN
1,1,3,0.003907,10102728,66888433_Comments_Event,4,03/06/2021 00:00,appeared relatively kempt,25,1,...,[],[],nan,nan,NaN,NaN,NaN,NaN,no pred (FN),FN


In [ ]:
# if only 1 label

def match_label(gold, pred):
    if pd.isna(pred) or pred.strip() == '' or pred=="nan":
        return "FN"
    elif pred == gold:
        return "TP"
    else:
        return "no match"

df['label_match'] = df.apply(lambda row: match_label(row['label'], row['label1']), axis=1)

In [274]:
# if 2 labels, update accordingly 

def match_label(gold, pred):
    if pd.isna(pred) or pred.strip() == '' or pred=="nan":
        return "FN"
    elif pred == gold == "3and1":
        return "TP"
    elif pred == gold == "3and0":
        return "TN"
    elif pred!= gold:
        return "FP"
    else:
        return "no match"

df['label_match'] = df.apply(lambda row: match_label(row['label'], row['label1']), axis=1)

In [275]:
df['label_match'].value_counts()

label_match
FN    219
TP     24
FP     16
TN      8
Name: count, dtype: int64

In [276]:
count_label_metrics = df['label_match'].value_counts()

In [277]:
TP = count_label_metrics.get('TP', 0)
FN = count_label_metrics.get('FN', 0)
FP = count_label_metrics.get('FP', 0)
TN = count_label_metrics.get('TN', 0)

print("TP: ", TP)
print("FN: ", FN)
print("FP: ", FP)
print("TN: ", TN)

TP:  24
FN:  219
FP:  16
TN:  8


In [278]:
precision = TP/(TP+FP)
recall = TP/(TP+FN)
f1 = 2*(precision*recall)/(precision+recall)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1)

Precision:  0.6
Recall:  0.09876543209876543
F1-score:  0.16961130742049468


In [279]:
SE_precision = math.sqrt((precision*(1-precision)/(TP+FP)))
SE_recall = math.sqrt((recall*(1-recall)/(TP+FN)))

print("Precision std error: ", SE_precision)
print("Recall std error: ", SE_recall)

Precision std error:  0.07745966692414834
Recall std error:  0.01913895705208628


In [280]:
z = 1.96

In [281]:
precision_lower_ci = precision - (1.96*SE_precision)
precision_upper_ci = precision + (1.96*SE_precision)

recall_lower_ci = recall - (1.96*SE_recall)
recall_upper_ci = recall + (1.96*SE_recall)

f1_lower_ci = 2*(precision_lower_ci*recall_lower_ci)/(precision_lower_ci+recall_lower_ci)
f1_upper_ci = 2*(precision_upper_ci*recall_upper_ci)/(precision_upper_ci+recall_upper_ci)

print("Precision lower CI: ", precision_lower_ci)
print("Precision upper CI: ", precision_upper_ci)
print("Recall lower CI: ", recall_lower_ci)
print("Recall lower CI: ", recall_upper_ci)
print("F1 lower CI: ", f1_lower_ci)
print("F1 lower CI: ", f1_upper_ci)

Precision lower CI:  0.4481790528286692
Precision upper CI:  0.7518209471713307
Recall lower CI:  0.06125307627667632
Recall lower CI:  0.13627778792085454
F1 lower CI:  0.10777626356914033
F1 lower CI:  0.23073221826497825


In [282]:
model_label_metrics = {
    'Metric': ['F1-score', 'Precision', 'Recall'],
    'Value': [f1, precision, recall],
    'Lower 95% CI': [f1_lower_ci, precision_lower_ci, recall_lower_ci],
    'Upper 95% CI': [f1_upper_ci, precision_upper_ci, recall_upper_ci]
}

In [283]:
df_label_metrics = pd.DataFrame(model_label_metrics)
df_label_metrics

,Metric,Value,Lower 95% CI,Upper 95% CI
0,F1-score,0.169611,0.107776,0.230732
1,Precision,0.600000,0.448179,0.751821
2,Recall,0.098765,0.061253,0.136278


In [284]:
df_label_metrics.to_csv('manual_eval_metrics_label_'+model+'_.csv')